This notebook provides an example of how to use the `poopy` package to access live event duration monitoring data provided by English water companies.

First, we import the libraries we need.

In [1]:
from poopy.companies import ThamesWater
# To help demonstrate the package
import time
import os
import matplotlib.pyplot as plt

The intended way to access active EDM data is by instantiating a `WaterCompany` object, which corresponds to the EDM sensor network maintained by a specific water comapny. When initialising the object, it is populated with all of the active (i.e., _transmitting_) EDM monitors maintained by Thames Water. However, we do not create a `WaterCompany` object directly. Instead, each water company defines a sub-class of a `WaterCompany` object. This is because each company transmits their data via different APIs and so the data is accessed in slightly different ways. However, this is all done 'behind the scenes', so they are all interacted with in the exact same way. The only difference is the name of the class. Lets have a look at the data in Thames Water's active EDM monitors...

To access the Thames Water API you need to specify access codes
for the API which you can obtain [here](https://data.thameswater.co.uk/s/). 
For security reasons, we have not included our access codes in this 
script. Instead, we assume that they have been set as *environment
variables*. If you are running this script on your own machine, you
will need to set these environment variables yourself. 

In [2]:
tw_clientID = os.getenv("TW_CLIENT_ID")
tw_clientSecret = os.getenv("TW_CLIENT_SECRET")

if tw_clientID is None or tw_clientSecret is None:
    raise ValueError(
        "Thames Water API keys are missing from the environment!\n Please set them and try again."
    )

tw = ThamesWater(tw_clientID, tw_clientSecret)

Initialising Thames Water object...
	Fetching model grid...
	This download can be slow if it is the first time you have generated a ThamesWater object. 
	The file will be cached so will be faster on subsequent uses...
Requesting current status data from Thames Water API...
	Requesting from https://prod-tw-opendata-app.uk-e1.cloudhub.io/data/STE/v1/DischargeCurrentStatus?limit=1000&offset=0
	Requesting from https://prod-tw-opendata-app.uk-e1.cloudhub.io/data/STE/v1/DischargeCurrentStatus?limit=1000&offset=1000
	No more records to fetch


We can see the names of these monitors by using the `active_monitor_names` attribute

In [3]:
print("-" * 50)
print("Number of active monitors: ", len(tw.active_monitor_names))
print("Active monitor names: ")
for name in tw.active_monitor_names:
    print("\t", name)
print("-" * 50)

--------------------------------------------------
Number of active monitors:  589
Active monitor names: 
	 (Northern) Low Level No 1 Brook Green
	 [Coming Soon] Abbeydale Road
	 [Coming Soon] Alicia Avenue CSO
	 [Coming Soon] Alperton Lane
	 [Coming Soon] Banbury Rd Middleton Cheney
	 [Coming Soon] Bookham SPS
	 [Coming Soon] Boyne Avenue, Hendon
	 [Coming Soon] Brackner Road CSO 
	 [Coming Soon] Brent Cross Flyover
	 [Coming Soon] Brent Valley Golf Course, LB Ealing
	 [Coming Soon] Bridle Close, Ewell
	 [Coming Soon] Bucknell
	 [Coming Soon] Bute Street, Outside No 43 (PH)
	 [Coming Soon] Carlton Cres Jnc Blenheim Cres
	 [Coming Soon] Channelsea West Ham Lane PS 
	 [Coming Soon] Chigwell Road / Langston Road
	 [Coming Soon] Chigwell Road, Woodford Green
	 [Coming Soon] Chisledon Storm Tanks 
	 [Coming Soon] Church Road
	 [Coming Soon] Church Road/Brent Lodge Park (Bunny Park)
	 [Coming Soon] Church Street KT13 8DP
	 [Coming Soon] Colin Crescent, Colindale
	 [Coming Soon] Conolly Road

Lets see the current status of a random monitor in the network. Lets extract a random monitor in the network

In [4]:
print("Selecting an arbitrary monitor...")
name = tw.active_monitor_names[359]
print("Monitor name: ", name)

Selecting an arbitrary monitor...
Monitor name:  Kempsford


The monitors are stored in a `Dictionary` of `Monitor` objects, which can be accessed using the `monitors` attribute. We now extract the `Monitor` object corresponding to the name we have just extracted. We can then use the `print_status` method to print the current status of the monitor.

In [5]:
monitor = tw.active_monitors[name]
monitor.print_status()


        
        --------------------------------------
        Event Type: Not Discharging
        Site Name: Kempsford
        Permit Number: CATM.3520
        OSGB Coordinates: (416570, 197540)
        Receiving Watercourse: Dudgrove Stream
        Start Time: 2023-12-02 17:00:00
        End Time: Ongoing
        Duration: 16985.68 minutes
        


Each `Monitor` stores the `WaterCompany` object which contains the monitor. For example, the `Monitor` we have just extracted is maintained by the following `WaterCompany` object:

In [6]:
print("Monitor maintained by: ", monitor.water_company.name)

Monitor maintained by:  ThamesWater


We can also see when the information was last updated by querying the WaterCompany object's timestamp attribute

In [7]:
print("Monitor data last updated: ", monitor.water_company.timestamp)

Monitor data last updated:  2023-12-14 12:05:40.441711


Lets say we think maybe there has been a change in the status of the monitor since the last update. We can use the `WaterCompany`'s `update()` method to update the status of the `Monitor`. Note that this updates all `Monitor`s maintained by the `WaterCompany` object, not just the one we are interested in.

In [8]:
print("... pausing for 5 seconds ...")
time.sleep(5)
print("Updating monitor data...")
monitor.water_company.update()
print("Monitor last updated: ", monitor.water_company.timestamp)

... pausing for 5 seconds ...
Updating monitor data...
Requesting current status data from Thames Water API...
	Requesting from https://prod-tw-opendata-app.uk-e1.cloudhub.io/data/STE/v1/DischargeCurrentStatus?limit=1000&offset=0
	Requesting from https://prod-tw-opendata-app.uk-e1.cloudhub.io/data/STE/v1/DischargeCurrentStatus?limit=1000&offset=1000
	No more records to fetch
Monitor last updated:  2023-12-14 12:05:46.386402


Note that the timestamp has been updated.

The `current_event` attribute of the `Monitor` object stores an `Event` object that contains specific information. `Event` is a class that contains three sub-classes corresponding to the three different types of status that can be recorded: `Discharge`, `NoDischarge` and `Offline`. The `current_event` attribute will contain an object of one of these three classes, depending on the current status of the monitor.

Lets see what the current status of the event recorded at the monitor is:

In [9]:
print("Extracting current event at monitor...")
event = monitor.current_event
print("Event ongoing? ", event.ongoing)

Extracting current event at monitor...
Event ongoing?  True


Because the event is ongoing it doesn't have an end time, but it does have a start time.
As a result, the `duration` attribute of the `Event` object updates dynamically to show the duration
of the event so far (in minutes). We can see this here.

In [10]:
print("Event start:", event.start_time)
print("Event duration:", event.duration, "minutes")
print("... pausing for 5 seconds ...")
time.sleep(5)
print("Event duration (5 seconds later):", event.duration, "minutes")

Event start: 2023-12-02 17:00:00
Event duration: 16985.773258616668 minutes
... pausing for 5 seconds ...
Event duration (5 seconds later): 16985.856661383335 minutes


Lets query all the CSOs that are currently discharging using the `WaterCompany`'s `discharging_monitors` attribute. This returns a list of `Monitor` objects that are currently discharging. We loop through and print their summaries. Note how the colour of the summary changes depending on the status of the monitor.

In [11]:
print("Extracting all discharging CSOs...")
discharging = tw.discharging_monitors
print("Printing summary of all discharging CSOs...")
for monitor in discharging:
    monitor.print_status()

Extracting all discharging CSOs...
Printing summary of all discharging CSOs...

        
        --------------------------------------
        Event Type: Discharging
        Site Name: Abbess Roding
        Permit Number: CTCR.2034
        OSGB Coordinates: (557580, 211090)
        Receiving Watercourse: Coopers Brook
        Start Time: 2023-12-03 09:30:00
        End Time: Ongoing
        Duration: 15995.86 minutes
        

        
        --------------------------------------
        Event Type: Discharging
        Site Name: Ampney St Peter
        Permit Number: CSSC.2452
        OSGB Coordinates: (408360, 200770)
        Receiving Watercourse: Ampney Brook
        Start Time: 2023-12-04 04:00:00
        End Time: Ongoing
        Duration: 14885.86 minutes
        

        
        --------------------------------------
        Event Type: Discharging
        Site Name: Andoversford
        Permit Number: CNTD.0001
        OSGB Coordinates: (402500, 219860)
        Receiving

Now we want to look at the downstream impact of the current sewage discharges. We do this using the `get_downstream_geojson` method of the `WaterCompany` object. This returns a `GeoJSON` object that contains as line features, the river sections that are downstream of the discharging CSOs. We optionally include the `include_recent_discharges` argument, which defaults to `False`. If this is `True`, then the `GeoJSON` object will also track the impact of discharges that are no longer ongoing but have occurred in the last 48 hours. 

In [12]:
geosjon = tw.get_downstream_geojson(include_recent_discharges = True)

Loading model grid from memory...
...can take a bit of time...
Building downstream geojson...
...can take a bit of time...


This object can be visualised in python using packages like, for example, geopandas. To use this information in other geospatial software, we can alternatively save
the downstream points as a file using the `save_downstream_geojson` (note that this calls `get_downstream_geojson`).
This is geoJSON line file that contains the downstream river sections.
This can be loaded into QGIS or other GIS software. By default, the file
is saved as a .geojson with a name concatenating the water company name
and the most recent update time. Note this function can be slow for large
networks. The output coordinate system is the British National Grid (epsg:27700).

In [ ]:
tw.save_downstream_geojson()